In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import re
import re
import sys
import os
import dotenv 
dotenv.load_dotenv()


key = os.getenv("token")

In [2]:
url = "https://tripadvisor16.p.rapidapi.com/api/v1/hotels/searchHotels"
querystring = {
    "geoId": "187438",
    "checkIn": "2024-11-01",
    "checkOut": "2024-11-03",
    "pageNumber": "1",
    "adults": "2",
    "currencyCode": "EUR"
}
headers = {
    "x-rapidapi-key": key,
    "x-rapidapi-host": "tripadvisor16.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
if response.status_code == 200:
    res = response.json()
    
    #como res es un diccionario que incluye dos elementos datas , entonces tenemos que entrar en el segundo data que es una
    #lista de diccionarios

    if 'data' in res and 'data' in res['data']: 
        hoteles = res['data']['data']

type(hoteles)


list

In [3]:
data = []
for hotel in hoteles:
        title = hotel.get('title')
            
        badge = hotel.get('badge', {})
        badge_size = badge.get('size')
        bubble_rating = hotel.get('bubbleRating', {})
        bubble_rating_value = bubble_rating.get('rating')
        price = hotel.get('priceForDisplay')
        provider = hotel.get('provider')
        card_photos = hotel.get('cardPhotos', [])
        commerce_info = hotel.get('commerceInfo', {})
        external_url = commerce_info.get('externalUrl')
        details = hotel.get('details', {})
        details_text = details.get('text')

        data.append({
                'Title': title,
                'Badge Size': badge_size,
                'Bubble Rating Value': bubble_rating_value,
                'Details Text': details_text,
                'Price': price,
                'Provider': provider,
                'External URL': external_url
                
            })
        
df = pd.DataFrame(data)


df.head(2)
  

,Title,Badge Size,Bubble Rating Value,Details Text,Price,Provider,External URL
0,1. ibis budget Malaga Aeropuerto Avenida Velaz...,None,3.5,None,€95,ibis Budget,https://www.tripadvisor.in/Commerce?p=Accor&sr...
1,2. Vincci Larios Diez,SMALL,5.0,None,None,Booking.com,None


In [5]:
type(res)   

dict

In [7]:
df.isnull().sum()

Title                   0
Badge Size              9
Bubble Rating Value     0
Details Text           30
Price                  29
Provider                3
External URL           29
dtype: int64

In [8]:
df_malaga = df.fillna('Nan')

In [9]:
df_malaga.columns = ['Nombre Hotel','Talla','Puntuacion media','Detalles extra','Precio','Proveedor','URL']

In [14]:
df_malaga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Nombre Hotel      30 non-null     object 
 1   Talla             30 non-null     object 
 2   Puntuacion media  30 non-null     float64
 3   Detalles extra    30 non-null     object 
 4   Precio            30 non-null     object 
 5   Proveedor         30 non-null     object 
 6   URL               30 non-null     object 
dtypes: float64(1), object(6)
memory usage: 1.8+ KB


In [18]:
df_malaga.to_csv('tripadvaisor_malaga.csv')

In [20]:
df_hoteles = pd.read_csv('../Datos/tripadvaisor_malaga.csv')
df_hoteles.head(2)

,Unnamed: 0,Nombre Hotel,Talla,Puntuacion media,Detalles extra,Precio,Proveedor,URL
0,0,1. ibis budget Malaga Aeropuerto Avenida Velaz...,Nan,3.5,Nan,€95,ibis Budget,https://www.tripadvisor.in/Commerce?p=Accor&sr...
1,1,2. Vincci Larios Diez,SMALL,5.0,Nan,Nan,Booking.com,Nan


In [21]:
df_hoteles.dtypes

Unnamed: 0            int64
Nombre Hotel         object
Talla                object
Puntuacion media    float64
Detalles extra       object
Precio               object
Proveedor            object
URL                  object
dtype: object

In [22]:
df_hoteles.drop(columns=['Unnamed: 0'], inplace=True)
df_hoteles['Precio'] = df_hoteles['Precio'].str.replace(',', '.')
df_hoteles['Precio'] = df_hoteles['Precio'].str.replace('€', '')

In [23]:
df_hoteles['Precio'] = df_hoteles['Precio'].astype(float)

In [24]:
df_hoteles.head(2)
df_hoteles.dtypes

Nombre Hotel         object
Talla                object
Puntuacion media    float64
Detalles extra       object
Precio              float64
Proveedor            object
URL                  object
dtype: object

In [25]:
df_hoteles.to_csv('../Datos/tripadvaisor_malaga.csv')